In [1]:
import sys
sys.path.append("../../.")

NThreads = 36

## Construction

In [2]:
from Construction.Function import Function
from Construction.Components import SBox
from Construction.CompoundFunction import CompoundFunction, INPUT_ID, OUTPUT_ID
from Construction.IteratedCipher import construct_iterated_cipher

In [3]:
# components
PRESENT_sbox = SBox(4, 4, [0xC, 5, 6, 0xB, 9, 0, 0xA, 0xD, 3, 0xE, 0xF, 8, 4, 7, 1, 2])
PRESENT_bitpermutation = [0, 16, 32, 48, 1, 17, 33, 49, 2, 18, 34, 50, 3, 19, 35, 51, 4, 20, 36, 52, 5, 21, 37, 53, 6, 22, 38, 54, 7, 23, 39, 55, 8, 24, 40, 56, 9, 25, 41, 57, 10, 26, 42, 58, 11, 27, 43, 59, 12, 28, 44, 60, 13, 29, 45, 61, 14, 30, 46, 62, 15, 31, 47, 63]

In [4]:
# construct round function
PRESENT_roundfunction = CompoundFunction(64, 64)
ids = []
# add sboxes
for _ in range(16):
    ids.append(PRESENT_roundfunction.add_component(PRESENT_sbox))
# connect input to components
for i in range(64):
    PRESENT_roundfunction.connect_components(INPUT_ID, i, ids[i//4], i%4)
# connect components to output
for i in range(64):
    PRESENT_roundfunction.connect_components(ids[i//4], i%4, OUTPUT_ID, PRESENT_bitpermutation[i])

## Analysis
First we enumerate all properties for 9 round PRESENT.
We'll try out different splits and merge their resulting bases in the end.
Then we try to find properties for 10 round PRESENT.

In [5]:
import os.path
import pickle
from Modelling.Search import search_integral_properties
from Tools.AvecImplementations import Avec_unified_model_with_partial_trail_counting_constant
from functools import reduce, partial
from pysat.formula import IDPool
from multiprocessing import Manager
from time import time

In [ ]:
# 9 rounds
splits = [(1, 7, 1), (1, 6, 2), (1, 5, 3), (2, 6, 1), (2, 5, 2), (2, 4, 3), (3, 5, 1), (3, 4, 2), (3, 3, 3)]
limit = 2**10
for split in splits:
    f1 = construct_iterated_cipher([PRESENT_roundfunction]*split[0], [2**64-1]*split[0] + [0])
    f2 = construct_iterated_cipher([PRESENT_roundfunction]*split[1], [2**64-1]*(split[1]+1))
    f3 = construct_iterated_cipher([PRESENT_roundfunction]*split[2], [0] + [2**64-1]*split[2])

    pool = IDPool()
    model, input_vars, intermediate_vars1, _ = f1.optimized_for_nonzero_trail_detection().to_model(pool)
    modelx, _, intermediate_vars2, _ = f2.optimized_for_nonzero_trail_detection().to_model(pool, input_vars=intermediate_vars1)
    model += modelx
    modelx, _, output_vars, _ = f3.optimized_for_nonzero_trail_detection().to_model(pool, input_vars=intermediate_vars2)
    model += modelx
    
    f1_model, f1_input_vars, f1_output_vars, f1_key_vars = f1.to_model()
    f2_model, f2_input_vars, f2_output_vars, f2_key_vars = f2.to_model()
    f3_model, f3_input_vars, f3_output_vars, f3_key_vars = f3.to_model()

    cachek1, cachek2, cachek3, cachec1, cachec2, cachec3, oracle_call_counts = Manager().dict(), Manager().dict(), Manager().dict(), Manager().dict(), Manager().dict(), Manager().dict(), Manager().list()

    Avec = partial(Avec_unified_model_with_partial_trail_counting_constant, cachek1, cachek2, cachek3, cachec1, cachec2, cachec3, oracle_call_counts, model, input_vars, intermediate_vars1, intermediate_vars2, output_vars, f1_model, f1_input_vars, f1_output_vars, f1_key_vars, f2_model, f2_input_vars, f2_output_vars, f2_key_vars, f3_model, f3_input_vars, f3_output_vars, f3_key_vars, limit=limit)

    fname = f"Results/R9-complex-oracle-{split[0]}-{split[1]}-{split[2]}.pkl"
    fname_stats = f"Results/R9-complex-oracle-{split[0]}-{split[1]}-{split[2]}-stats.txt"
    if os.path.isfile(fname):
        with open(fname, "rb") as f:
            res = pickle.load(f)
        with open(fname_stats, "r") as f:
            l = list(f.read().split(" ")[:2])
            total_time, oracle_call_count = float(l[0]), int(l[1])
    else:
        stime = time()
        res = search_integral_properties(Avec, 64, 64, True, NThreads)
        total_time = time() - stime
        oracle_call_count = sum(oracle_call_counts)
        with open(fname, "wb") as f:
            pickle.dump(res, f)
        with open(fname_stats, "w") as f:
            f.write(f"{total_time} {oracle_call_count}")
    print(split)
    print("time:", total_time)
    print("oracle_calls:", oracle_call_count)
    print("dimension:", len(res))
    print()

(1, 7, 1)
time: 8269.621418476105
oracle_calls: 2666126
dimension: 455

(1, 6, 2)
time: 13762.805321455002
oracle_calls: 19156473
dimension: 420

(1, 5, 3)
time: 2714.4336524009705
oracle_calls: 118593055
dimension: 338

(2, 6, 1)
time: 11397.889473438263
oracle_calls: 22921454
dimension: 425

(2, 5, 2)
time: 22672.75123310089
oracle_calls: 705298640
dimension: 401

(2, 4, 3)
time: 17749.31081867218
oracle_calls: 141070388201
dimension: 331

(3, 5, 1)
time: 3131.4764246940613
oracle_calls: 159686272
dimension: 338

(3, 4, 2)
time: 23786.157261371613
oracle_calls: 220841578547
dimension: 331



In [ ]:
splits = [(1, 8, 1), (1, 7, 2), (1, 6, 3), (2, 7, 1), (2, 6, 2), (2, 5, 3), (3, 6, 1), (3, 5, 2), (3, 4, 3)]
limit = 2**10
for split in splits:
    f1 = construct_iterated_cipher([PRESENT_roundfunction]*split[0], [2**64-1]*split[0] + [0])
    f2 = construct_iterated_cipher([PRESENT_roundfunction]*split[1], [2**64-1]*(split[1]+1))
    f3 = construct_iterated_cipher([PRESENT_roundfunction]*split[2], [0] + [2**64-1]*split[2])

    pool = IDPool()
    model, input_vars, intermediate_vars1, _ = f1.optimized_for_nonzero_trail_detection().to_model(pool)
    modelx, _, intermediate_vars2, _ = f2.optimized_for_nonzero_trail_detection().to_model(pool, input_vars=intermediate_vars1)
    model += modelx
    modelx, _, output_vars, _ = f3.optimized_for_nonzero_trail_detection().to_model(pool, input_vars=intermediate_vars2)
    model += modelx
    
    f1_model, f1_input_vars, f1_output_vars, f1_key_vars = f1.to_model()
    f2_model, f2_input_vars, f2_output_vars, f2_key_vars = f2.to_model()
    f3_model, f3_input_vars, f3_output_vars, f3_key_vars = f3.to_model()

    cachek1, cachek2, cachek3, cachec1, cachec2, cachec3, oracle_call_counts = Manager().dict(), Manager().dict(), Manager().dict(), Manager().dict(), Manager().dict(), Manager().dict(), Manager().list()

    Avec = partial(Avec_unified_model_with_partial_trail_counting_constant, cachek1, cachek2, cachek3, cachec1, cachec2, cachec3, oracle_call_counts, model, input_vars, intermediate_vars1, intermediate_vars2, output_vars, f1_model, f1_input_vars, f1_output_vars, f1_key_vars, f2_model, f2_input_vars, f2_output_vars, f2_key_vars, f3_model, f3_input_vars, f3_output_vars, f3_key_vars, limit=limit)

    fname = f"Results/R10-complex-oracle-{split[0]}-{split[1]}-{split[2]}.pkl"
    fname_stats = f"Results/R10-complex-oracle-{split[0]}-{split[1]}-{split[2]}-stats.txt"
    if os.path.isfile(fname):
        with open(fname, "rb") as f:
            res = pickle.load(f)
        with open(fname_stats, "r") as f:
            l = list(f.read().split(" ")[:2])
            total_time, oracle_call_count = float(l[0]), int(l[1])
    else:
        stime = time()
        res = search_integral_properties(Avec, 64, 64, True, NThreads)
        total_time = time() - stime
        oracle_call_count = sum(oracle_call_counts)
        with open(fname, "wb") as f:
            pickle.dump(res, f)
        with open(fname_stats, "w") as f:
            f.write(f"{total_time} {oracle_call_count}")
    print(split)
    print("time:", total_time)
    print("oracle_calls:", oracle_call_count)
    print("dimension:", len(res))
    print()